# Good pratices for loop optimisations : benefits of using numpy

using a for loop vs numpy for basic loop coding shows that you must avoid for loops in your code, here is a code snippet that shows the big performance difference to apply a square on 250 000 elements

In [19]:
import time
import numpy as np

start = time.perf_counter()
# Calculate square with for loop
array_forloop = []
for i in range(0,500):
    for j in range(0,500):
        array_forloop.append((i * j) ** 2)
    
end = time.perf_counter()
print("Time with for loop : {} ms".format((end-start)*1000))

# Using list comprehension instead of a for loop is a nice optimisation if your use case needs a for loop anyway
start = time.perf_counter()
array_forloop = [i**2 for i in range(0,250000)]
end = time.perf_counter()
print("Time with for list comprehension : {} ms".format((end-start)*1000))
      
# Using numpy
start = time.perf_counter()
arr = np.arange(250000)
squared_arr = np.square(arr)
end = time.perf_counter()
print("Time with numpy : {} ms".format((end-start)*1000))

Time with for loop : 83.00238009542227 ms
Time with for list comprehension : 63.063559122383595 ms
Time with numpy : 0.2981601282954216 ms


Conclusion

* Avoid using for loops to apply a computing on a bunch of pixels, use numpy instead
* If you need a for loop for another processing, use list comprehension as much as possible, it is very well documented 

For an in-depth comparison of how much numpy is faster than for and while loops, this link provides a complete performance comparison : https://www.blog.duomly.com/loops-in-python-comparison-and-performance/

# Good pratices for searching informations

If your code needs to search elements in an image for example looking at NoData pixels, it is highly recommended to do that search in a set instead of a list of pixels :

In [29]:
from timeit import timeit
import numpy as np

nd_list = [i for i in range(0,100000)]
nd_set = set([i for i in range(0,100000)])
nd_array = np.arange(0,100000)

#Look at an element
def search_list():
    if 98950 in nd_list:
        pass
    
#Look at the same element in the set
def search_set():
    if 98950 in nd_set:
        pass

def search_numpy():
    if np.where(nd_array==98950):
        pass

#lets say nodata value is 98950 in that case
t1 = timeit(search_list, number=100000)
t2 = timeit(search_set, number=100000)
t3 = timeit(search_numpy, number=100000)
print("List search time: {} s".format(t1))
print("Set search time: {} s".format(t2))
print("Numpy search time: {} s".format(t3))

List search time: 42.939479635097086 s
Set search time: 0.006291795987635851 s
Numpy search time: 4.1151342540979385 s


Conclusion

Use sets when you need to lookup for values in your image but beware that the set is not intended to be modified. You can alternatively use numpy.where but it is not as efficient as the set because it returns a boolean array for each element of the array, nevertheless it is 10 times more efficient than a list.

# A Green IT approach for coding image processing chains


## Downscaling rasters thanks to dask

In this notebook we will look at a green IT approach of coding, via good pratices in Python. We will have a look at Dask, RioXarray, rasterio, numpy... all these libraries that are widely used in the satellite image processing chains. 
In order to use dask we first need to create a local cluster.

### Creating the Dask Cluster

First let's import libraries needed for this tutorial and create our dask [LocalCluster](https://docs.dask.org/en/stable/deploying-python.html#localcluster) which allow us to create workers and use [dask's dashboard](https://docs.dask.org/en/latest/dashboard.html).


In [1]:
from pathlib import Path

from typing import List, Tuple, Union, Dict
import dask.array as da
import numpy as np
import rasterio
import rioxarray as rxr
from dask import delayed
from dask.distributed import Client, LocalCluster, Lock
from rasterio.transform import Affine

from utils import create_map_with_rasters

cluster = LocalCluster()
client = Client(cluster)

print("Dask Dashboard: ", client.dashboard_link)
client

/usr/local/lib/python3.10/dist-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40751 instead
  warnings.warn(


Dask Dashboard:  http://127.0.0.1:40751/status


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:40751/status,
Dashboard: http://127.0.0.1:40751/status,Workers: 4
Total threads: 8,Total memory: 56.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42135,Workers: 4
Dashboard: http://127.0.0.1:40751/status,Total threads: 8
Started: Just now,Total memory: 56.00 GiB
Comm: tcp://127.0.0.1:39021,Total threads: 2
Dashboard: http://127.0.0.1:44165/status,Memory: 14.00 GiB
Nanny: tcp://127.0.0.1:42625,


Dask return an url where the dashboard is availaible (usually http://127.0.0.1:8787/status). This is not a tutorial on how to use this dashboard, but we recommend using it in a separate window while using this notebook.

### Open raster thanks to rioxarray

Here we are going to open the raster data required for this tutorial, the RGB bands from a Sentinel-2 acquisition. To do this, we're going to use rioxarray and, more specifically, the [open_rasterio](https://corteva.github.io/rioxarray/html/rioxarray.html#rioxarray-open-rasterio) method, which opens the images lazily (without loading data into memory) and returns a `dask.array` object. 
From this method we will use the ``chunks`` and ``lock`` arguments, which respectively set a chunk size and limit access to the data to one thread at a time to avoid read problems. Here ``chunks`` is set to ``True`` to allow dask to automatically size chunks.


## Work directories

In [13]:
sentinel_2_dir = "/work/scratch/data/romaint"
s2_b4 = f"{sentinel_2_dir}/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1_FRE_B4.tif"
s2_b8 = f"{sentinel_2_dir}/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1_FRE_B8.tif"
phr_product = "/work/scratch/data/tanguyy/public/PHR_OTB/Marmande/IMG_PHR1A_PMS_202304151100243_SEN_6967638101-1_R1C1_wnodata.tif"
phr_product_cog = "/work/scratch/data/romaint/phr_cog.tif"

# Calculation of the Average NDVI on a satellite image

In this example, we will use what we have learned to: 

1. Read the data from the disk and stack them.
2. Calculate the associated NDVI, which combines multi-band information into a single band.
3. Reduce the information by calculating the average NDVI within a window.
4. Write the resulting image to the disk.

First, let's read the data we need to perform the NDVI.

### Defining usefull functions for our notebook

In [3]:
def open_raster_and_get_metadata(raster_paths: List[str], chunks: Union[int, Tuple, Dict, None]):
    """
    Opens multiple raster files, extracts shared geospatial metadata, 
    and returns the concatenated data along with resolution and CRS info.

    Parameters:
    -----------
    raster_paths : List[str]
        Paths to the raster files.
    chunks : Union[int, Tuple, Dict, bool, None]
        Chunk sizes for Dask (bands, height, width).

    Returns:
    --------
    Tuple[dask.array.Array, float, float, float, float, Union[str, CRS]]
        Concatenated raster data, x and y resolution, top-left coordinates, and CRS.
    """
    results = []
    for raster_path in raster_paths:
        with rxr.open_rasterio(raster_path, chunks=chunks, lock=True) as tif:
            reprojection = tif
            transform = reprojection.rio.transform()
            crs = reprojection.rio.crs
            x_res = transform[0]
            y_res = -transform[4]
            top_left_x = transform[2]
            top_left_y = transform[5]
            results.append(reprojection)

    return da.concatenate(results), x_res, y_res, top_left_x, top_left_y, crs

def create_raster(data: np.ndarray, output_file: Path, x_res, y_res, top_left_x, top_left_y, crs):
    transform = Affine.translation(top_left_x, top_left_y) * Affine.scale(x_res, -y_res)
    with rasterio.open(
            output_file, "w",
            driver="GTiff",
            height=data.shape[1],
            width=data.shape[2],
            count=data.shape[0],
            dtype=data.dtype,
            crs=crs,
            transform=transform
    ) as dst:
        dst.write(data)


When the data is read, we can express the NDVI calculation as if it were a numpy array. We add ``[None, :, :]`` to keep the shape as ``(bands, rows, cols)``. Then we can apply reduction on the dask.array and use ``compute()`` on it to triger the computation.

In [13]:
# Open the raster
# Define our own chunks for better performance
reading_chunks = (-1,2200,2200)
#reading_chunks = True
input_data_array, x_res, y_res, top_left_x, top_left_y, crs = open_raster_and_get_metadata([s2_b4,s2_b8], reading_chunks)
input_data_array

dask.array<concatenate, shape=(2, 10980, 10980), dtype=int16, chunksize=(1, 2200, 2200), chunktype=numpy.ndarray>

In [12]:
%%time

ndvi_array = (input_data_array[1] - input_data_array[0]) / (input_data_array[1] + input_data_array[0])[None, :, :]
# Launch the computing with dask with the compute call
mean_ndvi = ndvi_array.compute() 
crs="EPSG:4326"
output_file = Path("/work/scratch/data/romaint/output_greenit/ndvi_dask.tif")
create_raster(mean_ndvi, output_file, x_res , y_res,
                  top_left_x, top_left_y, crs)

CPU times: user 528 ms, sys: 2.08 s, total: 2.61 s
Wall time: 6.19 s


## Calculate NDVI With OTB in python

In [12]:
import otbApplication as otb

out_ndvi_otb_py="/work/scratch/data/romaint/output_greenit/img_ndvi_otb_py.tif"
#Compute NDVI with OTB in python
app_ndvi_otb = otb.Registry.CreateApplication("BandMath")
app_ndvi_otb.SetParameterStringList("il",[s2_b4,s2_b8])
app_ndvi_otb.SetParameterString("exp","(im2b1-im1b1)/(im2b1+im1b1)")
#app_ndvi_otb.SetParameterStringList("il",[phr_product])
#app_ndvi_otb.SetParameterString("exp","(im1b4-im1b1)/(im1b4+im1b1)")
app_ndvi_otb.SetParameterString("out",out_ndvi_otb_py)
app_ndvi_otb.ExecuteAndWriteOutput()

Warning 1: Invalid value for NUM_THREADS: 


Writing /work/scratch/data/romaint/img_ndvi_otb.tif...: 100% [**************************************************] (4s)


0

## Calculate NDVI with OTB in C++
This part will call BandMath with the otb CLI to compare performances with the python swig interface

In [ ]:
%%bash

otbcli_BandMath -il "/work/scratch/data/romaint/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1_FRE_B4.tif" "/work/scratch/data/romaint/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1_FRE_B8.tif" -exp "( im2b1 - im1b1 ) / ( im2b1 + im1b1 )" -out "/work/scratch/data/romaint/output_greenit/img_ndvi_otb_cpp.tif" 

Warning 1: Invalid value for NUM_THREADS: 


Writing /work/scratch/data/romaint/img_ndvi_otb_cpp.tif...: 100% [**************************************************] (4s)


# Optimizing the computing of NDVI

Here we will try to use numba for ndvi computation

In [16]:
%%time

from numba import jit,njit
from xarray import DataArray 
from typing import List, Tuple, Union, Dict
import rioxarray as rxr
import numpy as np
import time
import rasterio
from pathlib import Path
import xarray

@njit
def one_pixel_ndvi(p1,p2):
    return (p2-p1) / (p2+p1) 

@njit
def compute_ndvi_numba(input_data_1: np.ndarray,input_data_2: np.ndarray):
    #ndvi_array = [one_pixel_ndvi(i,j) for i in input_data_1 for j in input_data_2]
    ndvi_array = (input_data_2 - input_data_1) / (input_data_2 + input_data_1)
    return ndvi_array

def compute_ndvi_dask(input_data_1: DataArray,input_data_2: DataArray):
    ndvi_array = ((input_data_2 - input_data_1) / (input_data_2 + input_data_1))[None,:,:]
    ndvi_array.compute()
    return ndvi_array

def compute_ndvi_std(input_data_1: np.ndarray,input_data_2: np.ndarray):
    #ndvi_array = [one_pixel_ndvi(i,j) for i in input_data_1 for j in input_data_2]
    ndvi_array = (input_data_2 - input_data_1) / (input_data_2 + input_data_1)
    return ndvi_array

sentinel_2_dir = "/work/scratch/data/romaint"
s2_b4 = f"{sentinel_2_dir}/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1_FRE_B4.tif"
s2_b8 = f"{sentinel_2_dir}/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1/SENTINEL2B_20240822-105857-973_L2A_T31TCJ_C_V3-1_FRE_B8.tif"
phr_product = "/work/scratch/data/tanguyy/public/PHR_OTB/Marmande/IMG_PHR1A_PMS_202304151100243_SEN_6967638101-1_R1C1_wnodata.tif" 

start = time.perf_counter()
with rasterio.open(s2_b4, 'r') as ds:
    input_data_b4 = ds.read() 

with rasterio.open(s2_b8, 'r') as ds:
    input_data_b8 = ds.read()
   
ndvi_computed = compute_ndvi_numba(input_data_b4,input_data_b8)
crs="EPSG:4326"
output_file = Path("/work/scratch/data/romaint/output_greenit/ndvi_numba.tif")
create_raster(ndvi_computed, output_file, x_res , y_res, top_left_x, top_left_y, crs)
end = time.perf_counter()
print("Elapsed with Raster IO + numba = {}s".format((end - start)))

start = time.perf_counter()
with rasterio.open(s2_b4, 'r') as ds:
    input_data_b4 = ds.read() 

with rasterio.open(s2_b8, 'r') as ds:
    input_data_b8 = ds.read()

ndvi_computed = compute_ndvi_std(input_data_b4,input_data_b8)
crs="EPSG:4326"
output_file = Path("/work/scratch/data/romaint/output_greenit/ndvi_without_numba.tif")
create_raster(ndvi_computed, output_file, x_res , y_res,top_left_x, top_left_y, crs)
end = time.perf_counter()
print("Elapsed with Raster IO + without numba = {}s".format((end - start)))
# Example with xarray
start = time.perf_counter()
input_data_b4 =  xarray.open_dataarray(s2_b4)
input_data_b8 =  xarray.open_dataarray(s2_b8)
ndvi_computed = xarray.apply_ufunc(compute_ndvi_std,input_data_b4,input_data_b8)
ndvi_computed.rio.to_raster("/work/scratch/data/romaint/output_greenit/ndvi_ufunc.tif")
output_file = Path("/work/scratch/data/romaint/output_greenit/ndvi_ufunc.tif")
end = time.perf_counter()
print("Elapsed with Xarray + apply ufunc = {}s".format((end - start)))

start = time.perf_counter()
input_data_b4 = rxr.open_rasterio(s2_b4,chunks=True)
input_data_b8 = rxr.open_rasterio(s2_b8,chunks=True)
ndvi_array = (input_data_b8 - input_data_b4) / (input_data_b8 + input_data_b4)
ndvi_array.compute()
output_file = Path("/work/scratch/data/romaint/output_greenit/ndvi_std.tif")
ndvi_array.rio.to_raster(output_file)
end = time.perf_counter()
print("Elapsed with RIOXarray + dask = {}s".format((end - start)))

#phr_product = "/work/scratch/data/tanguyy/public/PHR_OTB/Marmande/IMG_PHR1A_PMS_202304151100243_SEN_6967638101-1_R1C1_wnodata.tif"
#phr_product = "/work/scratch/data/romaint/phr_cog.tif"
#start = time.perf_counter()
#reading_chunks = True
#input_data_array = rxr.open_rasterio(phr_product,chunks=reading_chunks,lock=False)
#print(input_data_array.shape)
#ndvi_phr = (input_data_array[3] - input_data_array[0]) / (input_data_array[0] + input_data_array[3])
#ndvi_phr.compute()
#print(ndvi_phr.shape)
#output_file = Path("/work/scratch/data/romaint/output_greenit/ndvi_phr.tif")
#ndvi_phr.rio.to_raster(output_file,tiled=True)
#end = time.perf_counter()
#print("Elapsed with RIOXarray + dask + big product = {}s".format((end - start)))

Elapsed with Raster IO + numba = 1.2573514231480658s
Elapsed with Raster IO + without numba = 1.6734004551544785s
Elapsed with Xarray + apply ufunc = 2.9065667972899973s
Elapsed with RIOXarray + dask = 6.7410209202207625s
CPU times: user 2.53 s, sys: 8.62 s, total: 11.1 s
Wall time: 12.6 s


## Differences between using Compute vs using RIO tiled write

## Optimize the chunk size for dask

Chunk size is becoming very important when your data size grows. You can let chunks=True to dask which will automatically determine a chunk size
Most of the time this chunk size is coherent but users can tweak it to be more efficient.

Also be careful about the compute() method, which is not recommended when your data size => 10Go, the memory consumption increases a lot

In [18]:
# First example letting dask compute the chunks
reading_chunks = True
data_array_autochunks = rxr.open_rasterio(phr_product_cog, chunks=reading_chunks,lock=False)
data_array_autochunks

<xarray.DataArray (band: 4, y: 41663, x: 39844)> Size: 13GB
dask.array<open_rasterio-fc2e18048eec7fd8788f8fc93c7ebea6<this-array>, shape=(4, 41663, 39844), dtype=uint16, chunksize=(1, 8192, 8192), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 32B 1 2 3 4
  * x            (x) float64 319kB 0.5 1.5 2.5 ... 3.984e+04 3.984e+04 3.984e+04
  * y            (y) float64 333kB 0.5 1.5 2.5 ... 4.166e+04 4.166e+04 4.166e+04
    spatial_ref  int64 8B 0
Attributes: (12/29)
    AcquisitionDate:      2023-04-15T11:00:24.3Z
    BlueDisplayChannel:   0
    DataType:             3
    GeometricLevel:       SENSOR
    GreenDisplayChannel:  0
    ImageID:              6967638101-1
    ...                   ...
    TimeRangeEnd:         2023-04-15T11:00:27.3815980Z
    TimeRangeStart:       2023-04-15T11:00:24.3193675Z
    NoData:               0
    _FillValue:           0
    scale_factor:         1.0
    add_offset:           0.0

In [20]:
import time
import rioxarray as rxr

start=time.perf_counter()
ndvi_array = (data_array_autochunks[3] - data_array_autochunks[0]) / (data_array_autochunks[3] + data_array_autochunks[0])
#mean_ndvi = ndvi_array.compute()
output_file = Path("/work/scratch/data/romaint/output_greenit/ndvi_autochunks.tif")
ndvi_array.rio.to_raster(output_file,tiled=True)
end=time.perf_counter()
print("Elapsed with automatic chunk size = {}s".format((end - start)))

/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:366: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  dataset = writer(


Elapsed with automatic chunk size = 57.985147991683334s


In [23]:
reading_chunks = (-1,2048,2048)
data_array_manualchunks = rxr.open_rasterio(phr_product_cog, chunks=reading_chunks,lock=False)
data_array_manualchunks

<xarray.DataArray (band: 4, y: 41663, x: 39844)> Size: 13GB
dask.array<open_rasterio-171af4db274709fd77ecac80bd238d05<this-array>, shape=(4, 41663, 39844), dtype=uint16, chunksize=(4, 2048, 2048), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 32B 1 2 3 4
  * x            (x) float64 319kB 0.5 1.5 2.5 ... 3.984e+04 3.984e+04 3.984e+04
  * y            (y) float64 333kB 0.5 1.5 2.5 ... 4.166e+04 4.166e+04 4.166e+04
    spatial_ref  int64 8B 0
Attributes: (12/29)
    AcquisitionDate:      2023-04-15T11:00:24.3Z
    BlueDisplayChannel:   0
    DataType:             3
    GeometricLevel:       SENSOR
    GreenDisplayChannel:  0
    ImageID:              6967638101-1
    ...                   ...
    TimeRangeEnd:         2023-04-15T11:00:27.3815980Z
    TimeRangeStart:       2023-04-15T11:00:24.3193675Z
    NoData:               0
    _FillValue:           0
    scale_factor:         1.0
    add_offset:           0.0

In [24]:
import time
import rioxarray as rxr

start=time.perf_counter()
ndvi_array = (data_array_manualchunks[3] - data_array_manualchunks[0]) / (data_array_manualchunks[3] + data_array_manualchunks[0])
#mean_ndvi = ndvi_array.compute()
output_file = Path("/work/scratch/data/romaint/output_greenit/ndvi_manualchunks.tif")
#create_raster(mean_ndvi, output_file, x_res , y_res, top_left_x, top_left_y, crs)
ndvi_array.rio.to_raster(output_file,tiled=True)
end=time.perf_counter()
print("Elapsed with manual chunk size = {}s".format((end - start)))

/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:366: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  dataset = writer(


Elapsed with manual chunk size = 45.831558969803154s


Conclusion

When using multiple band products, dask will take chunks per band, which slows done the processing in case of a processing using more than one band. We recommand in that case defining chunks using "(-1,sizex,sizey)". The time gain can go about 20% !

In our case we used chunks of 32Mo, it is a best practice to use chunks size multiple of 8 for read/write efficiency in RAM.

# Parallel write with dask vs multithreading only 

In [4]:
%%time

from xarray import DataArray 
from typing import List, Tuple, Union, Dict
import rioxarray as rxr
import numpy as np
import time
import rasterio
from pathlib import Path
import xarray

phr_product = "/work/scratch/data/romaint/phr_cog.tif"
start = time.perf_counter()
reading_chunks = True
input_data_array = rxr.open_rasterio(phr_product,chunks=reading_chunks,lock=False)
print(input_data_array.shape)
ndvi_phr = (input_data_array[3] - input_data_array[0]) / (input_data_array[0] + input_data_array[3])
#ndvi_phr.compute()
print(ndvi_phr.shape)
output_file = Path("/work/scratch/data/romaint/output_greenit/ndvi_phr.tif")
ndvi_phr.rio.to_raster(output_file,tiled=True)
end = time.perf_counter()
print("Elapsed with RIOXarray + dask + big product = {}s".format((end - start)))

(4, 41663, 39844)
(41663, 39844)


/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:366: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  dataset = writer(


Elapsed with RIOXarray + dask + big product = 55.51729126833379s
CPU times: user 1min 42s, sys: 28.8 s, total: 2min 11s
Wall time: 55.5 s


In [19]:
input_data_array

dask.array<concatenate, shape=(2, 10980, 10980), dtype=int16, chunksize=(1, 2200, 2200), chunktype=numpy.ndarray>

In [1]:
from xarray import DataArray 
from typing import List, Tuple, Union, Dict
import rioxarray as rxr
import numpy as np
import time
import rasterio
from pathlib import Path
import xarray

#phr_product = "/work/scratch/data/romaint/phr_cog.tif"
phr_product = "/work/scratch/data/tanguyy/public/PHR_OTB/Marmande/IMG_PHR1A_PMS_202304151100243_SEN_6967638101-1_R1C1_wnodata.tif"
reading_chunks = (-1,8192,8192)
input_data_array = rxr.open_rasterio(phr_product,chunks=reading_chunks)
#input_data_array, x_res, y_res, top_left_x, top_left_y, crs = open_raster_and_get_metadata([phr_product], reading_chunks)

In [2]:
input_data_array

<xarray.DataArray (band: 4, y: 41663, x: 39844)> Size: 13GB
dask.array<open_rasterio-e809baba0b7130783671e0472b541e4c<this-array>, shape=(4, 41663, 39844), dtype=uint16, chunksize=(4, 8192, 8192), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 32B 1 2 3 4
  * x            (x) float64 319kB 0.5 1.5 2.5 ... 3.984e+04 3.984e+04 3.984e+04
  * y            (y) float64 333kB 0.5 1.5 2.5 ... 4.166e+04 4.166e+04 4.166e+04
    spatial_ref  int64 8B 0
Attributes: (12/29)
    AcquisitionDate:      2023-04-15T11:00:24.3Z
    BlueDisplayChannel:   0
    DataType:             3
    GeometricLevel:       SENSOR
    GreenDisplayChannel:  0
    ImageID:              6967638101-1
    ...                   ...
    TimeRangeEnd:         2023-04-15T11:00:27.3815980Z
    TimeRangeStart:       2023-04-15T11:00:24.3193675Z
    NoData:               0
    _FillValue:           0
    scale_factor:         1.0
    add_offset:           0.0

In [3]:
start = time.perf_counter()
ndvi_phr = (input_data_array[3] - input_data_array[0]) / (input_data_array[3] + input_data_array[0])
ndvi_phr.compute()
print(ndvi_phr.shape)
output_file = Path("/work/scratch/data/romaint/output_greenit/ndvi_phr.tif")
#create_raster(ndvi_computed, output_file, x_res , y_res, top_left_x, top_left_y, crs)
ndvi_phr.rio.to_raster(output_file)
end = time.perf_counter()
print("Elapsed with RIOXarray + dask + big product with write = {}s".format((end - start)))


/usr/local/lib/python3.10/dist-packages/dask/_task_spec.py:651: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)


(41663, 39844)


/usr/local/lib/python3.10/dist-packages/rasterio/__init__.py:366: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  dataset = writer(


Elapsed with RIOXarray + dask + big product with write = 182.01451965374872s


/usr/local/lib/python3.10/dist-packages/dask/_task_spec.py:651: RuntimeWarning: invalid value encountered in divide
  return self.func(*new_argspec)


# Conclusions and recommandations about parallel write


# Performances multiprocessing example


# Optimizing the size of your data

In [ ]:
using COG ?

# Estimate the carbon impact of your code
Using code carbon you can have an estimate of your code footprint

In [6]:
from codecarbon import track_emissions


ModuleNotFoundError: No module named 'codecarbon'